<a href="https://colab.research.google.com/github/AbrarAhmed647/CarZam--A-Vehicular-Classification-using-EDNA-ML/blob/main/Fall_'22_CarZam_Part_I.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Downloading

In [1]:
import os
if not os.path.exists('./original_tool_image.zip'):
  ! wget -O original_tool_image.zip https://www.dropbox.com/s/pha9yzdfkmzoqob/original_tool_images.zip?dl=0

--2022-12-02 03:21:29--  https://www.dropbox.com/s/pha9yzdfkmzoqob/original_tool_images.zip?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.3.18, 2620:100:601b:18::a27d:812
Connecting to www.dropbox.com (www.dropbox.com)|162.125.3.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/raw/pha9yzdfkmzoqob/original_tool_images.zip [following]
--2022-12-02 03:21:29--  https://www.dropbox.com/s/raw/pha9yzdfkmzoqob/original_tool_images.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc077de2f19b70b9e4f691a669a7.dl.dropboxusercontent.com/cd/0/inline/Bx3kUNjZwv-sxFEXEjBTjMThUK4zXSeomdThkyQyx7_CSjgRQbXlp6UTgswbTtJA5TFJVVpTgpWQa5CXSDtlKYxsVLG90eNqQJqEsZUpv9nT-0dRO_NI4Pnttsmk5WVVjVRAVNyIrK_W5TnImMh1GoDws90MJVUXPQKqKnNQvuym9A/file# [following]
--2022-12-02 03:21:30--  https://uc077de2f19b70b9e4f691a669a7.dl.dropboxusercontent.com/cd/0/inline/Bx3kUNjZwv-sxFEXEjBTjMThUK4zXSeomdThkyQ

# Setup

## Git Clone

### From Source

In [2]:
! rm -rf -- GLAMOR

In [3]:
! git clone -b master https://github.com/asuprem/GLAMOR

Cloning into 'GLAMOR'...
remote: Enumerating objects: 8407, done.
remote: Counting objects: 100% (310/310), done.
remote: Compressing objects: 100% (171/171), done.
remote: Total 8407 (delta 187), reused 214 (delta 106), pack-reused 8097
Receiving objects: 100% (8407/8407), 2.21 MiB | 13.64 MiB/s, done.
Resolving deltas: 100% (5580/5580), done.


In [4]:
!pip install -e GLAMOR/

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Obtaining file:///content/GLAMOR
     |████████████████████████████████| 1.3 MB 7.4 MB/s 
  Running setup.py develop for ednaml


###  From PyPi

In [5]:
#! python -V

In [6]:
#! pip3 install --pre ednaml==0.1.4

# Restart Notebook to Finish EdnaML Installation

# Setting up

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
import ednaml
import glob, os
#from ednaml.core import EdnaDeploy, EdnaML
torch.__version__

'1.12.1+cu113'

## Definitions: Crawler

In [3]:
# Here we define our custom model class
from ednaml.crawlers import Crawler
from zipfile import ZipFile # might be useful in unzipping!

class CarZamCrawler(Crawler):
  def __init__(self, logger, file_name = "original_tool_images.zip", **kwargs): # Add your own arguments if needed!
    self.classes = {}
    self.metadata = {}
    self.metadata["train"] = {}
    self.metadata["test"] = {}
    self.metadata["val"] = {}
    self.metadata["train"]["crawl"] = []  # <------ THIS NEEDS TO BE POPULATED
    self.metadata["test"]["crawl"] = []   # <------ THIS NEEDS TO BE POPULATED
    self.metadata["val"]["crawl"] = []    # <------ THIS NEEDS TO BE POPULATED

    # YOUR CODE HERE ------ POPULATE self.classes and self.metadata's empty lists ---
    from zipfile import ZipFile
    file_name = "original_tool_image.zip"
    fdest= "unzipped"
    if not os.path.exists(fdest):
      with ZipFile(file_name, 'r') as zip: 
          # extract all files to another directory
          zip.extractall(fdest)
    fllist = glob.glob(os.path.join(fdest, "original_tool_images/*.jpg"))
    #tuple_prelim = [self.getinittuple(item) for item in fllist]
    temp=[]
    #ans=[]
    tokeep = ["Convertible", "Coupe", "Crossover", "Diesel", "Hybrid", "Sedan", "SUV", "Wagon","SportsCar", "Truck", "Van", ]
    tuple_expanded=[]
    for item in fllist:
      tuple_prelim=os.path.splitext(os.path.basename(item))[0].split(" "), item
      #print(tuple_prelim)
      if(len(tuple_prelim[0])==5):
        temp.append(tuple_prelim[1]) #appending path
        for i in tuple_prelim[0]:  #appending type,color,year,make,model
            if(i=="CoupeBlack"):
             temp.append("Coupe")
             temp.append("Black")
             temp.append(tuple_prelim[1:])
            else:
              temp.append(i)
        
        my_tuple=tuple(temp)        #list to Tuple
        tuple_expanded.append(my_tuple)   
        temp=[]
      elif(len(tuple_prelim[0])==6):
        temp.append(tuple_prelim[1])
        if(tuple_prelim[0][0]) in tokeep:
          for i in tuple_prelim[0][:4]:
            temp.append(i)
          temp.append(tuple_prelim[0][4]+tuple_prelim[0][5])

          my_tuple=tuple(temp)
          tuple_expanded.append(my_tuple)
          temp=[]
        else:
          temp.append(tuple_prelim[0][0]+tuple_prelim[0][1])
          for i in tuple_prelim[0][2:]:
            temp.append(i)
          #temp.append(tuple_prelim[1])
          my_tuple=tuple(temp)
          tuple_expanded.append(my_tuple)
          temp=[]
  #print(ans)
      elif(len(tuple_prelim[0])==7):
        temp.append(tuple_prelim[1])
        if tuple_prelim[0][0] in tokeep:
          for i in tuple_prelim[0][:4]:
            temp.append(i)
          temp.append(tuple_prelim[0][4:])

          my_tuple=tuple(temp)
          tuple_expanded.append(my_tuple)
          temp=[]
        else:
          temp.append(tuple_prelim[0][0]+tuple_prelim[0][1])
          for i in tuple_prelim[0][2:5]:
            temp.append(i)
          temp.append(tuple_prelim[0][5]+tuple_prelim[0][6])
        
          my_tuple=tuple(temp)
          tuple_expanded.append(my_tuple)
          temp=[]
    #print("Tuple_expanded:")
    #print(tuple_expanded[0])
    import random
    random.seed(3456)
    random.shuffle(tuple_expanded)

    splits = 0.8
    train_sets = int(len(tuple_expanded)*0.8)
    val_sets = int(len(tuple_expanded)*0.1)

    

    # structure:  (path, type, color, year, make)
    # idx           0     1     2     3     4
    print("Outliers\n")
    for item in tuple_expanded:
      if(item[1]=='CoupeBlack'):
        print(item)
        print("\n")
    
    types = list(set([item[1] for item in tuple_expanded]))
    print("types:",types)
    colors = list(set([item[2] for item in tuple_expanded]))
    years = list(set([item[3] for item in tuple_expanded]))
    makes = list(set([item[4] for item in tuple_expanded]))

    self.classes["vtype"] = len(types)
    self.classes["color"] = len(colors)
    self.classes["year"] = len(years)
    self.classes["make"] = len(makes)

    self.type_lookup = {item:idx for idx,item in enumerate(types)}
    self.color_lookup = {item:idx for idx,item in enumerate(colors)}
    self.year_lookup = {item:idx for idx,item in enumerate(years)}
    self.make_lookup = {item:idx for idx,item in enumerate(makes)}

    tuple_expanded = [(item[0], self.type_lookup[item[1]], self.color_lookup[item[2]], self.year_lookup[item[3]], self.make_lookup[item[4]]) for item in tuple_expanded]
    print("Tuple_expanded\n")
    print(tuple_expanded[:5])

    #split the datasets
    self.metadata["train"]["crawl"] = tuple_expanded[:train_sets]
    self.metadata["val"]["crawl"] = tuple_expanded[train_sets:val_sets]
    self.metadata["test"]["crawl"] = tuple_expanded[train_sets+val_sets:]


    # -------------------------------------------------------------------------------

    self.metadata["train"]["classes"] = self.classes
    self.metadata["test"]["classes"] = self.classes
    self.metadata["val"]["classes"] = self.classes

  def getinittuple(self, item):
    return (os.path.splitext(os.path.basename(item))[0].split(" "), item)

    


## Testing the Crawler

In [5]:
kwargs = {
    "logger" : None,
    "file_name" : "original_tool_images.zip",
    # add any other kwargs here...
}

In [6]:
crawler = CarZamCrawler(**kwargs)

Outliers

types: ['ElectricVehicle', 'Wagon', 'Coupe', 'SUV', 'Van', 'PickupTruck', 'Sedan', 'Hybrid', 'LuxuryVehicle', 'Convertible', 'Diesel', 'SportsCar', 'Crossover']
Tuple_expanded

[('unzipped/original_tool_images/Hybrid Silver 2021 Toyota Avalon Hybrid.jpg', 7, 2, 0, 7), ('unzipped/original_tool_images/Crossover Black 2018 Jeep Compass.jpg', 12, 3, 5, 27), ('unzipped/original_tool_images/Sedan Gray 2017 Honda Fit.jpg', 6, 0, 8, 10), ('unzipped/original_tool_images/Pickup Truck Tan 2017 Toyota Tacoma.jpg', 5, 7, 8, 7), ('unzipped/original_tool_images/Luxury Vehicle Gold 2015 Cadillac CTS.jpg', 8, 11, 1, 18)]


In [7]:
crawler.classes # You should get the classes here

{'vtype': 13, 'color': 13, 'year': 14, 'make': 36}

In [8]:
crawler.metadata["train"]["crawl"][:5]  # You should get the list of tuples here

[('unzipped/original_tool_images/Hybrid Silver 2021 Toyota Avalon Hybrid.jpg',
  7,
  2,
  0,
  7),
 ('unzipped/original_tool_images/Crossover Black 2018 Jeep Compass.jpg',
  12,
  3,
  5,
  27),
 ('unzipped/original_tool_images/Sedan Gray 2017 Honda Fit.jpg', 6, 0, 8, 10),
 ('unzipped/original_tool_images/Pickup Truck Tan 2017 Toyota Tacoma.jpg',
  5,
  7,
  8,
  7),
 ('unzipped/original_tool_images/Luxury Vehicle Gold 2015 Cadillac CTS.jpg',
  8,
  11,
  1,
  18)]

# Statistics

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
# Write code to collect info on # makes, models, year, type

3.1 Single classification (Vehicle Type)

In [9]:
class_name = "vtype"  # Make sure to change this to whatever name you used for make in your `original_tool_images` crawler
class_idx = 1         # Make sure to change this to whetever index `type` is in your Crawler's tuple!
path_idx = 0          # Change this to whichever index in tuple has path
crawler_args = {"file_name" : "original_tool_image.zip"}

In [10]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
from ednaml.core import EdnaML
from ednaml.generators import ClassificationGenerator

eml = EdnaML(config = "./GLAMOR/profiles/CarZam/base_config.yml", config_inject=[
    ("SAVE.MODEL_QUALIFIER", class_name)
])

eml.cfg.EXECUTION.DATAREADER.CRAWLER_ARGS = crawler_args
eml.cfg.EXECUTION.DATAREADER.DATASET_ARGS["pathidx"] = path_idx
eml.cfg.EXECUTION.DATAREADER.DATASET_ARGS["annotationidx"] = class_idx
eml.cfg.EXECUTION.DATAREADER.DATASET_ARGS["classificationclass"] = class_name

eml.addGeneratorClass(ClassificationGenerator)
eml.addCrawlerClass(CarZamCrawler)

Injected key-value pair:  SAVE.MODEL_QUALIFIER, vtype


In [12]:
eml.apply()

03:26:57 ****************************************
03:26:57 
03:26:57 
03:26:57 Using the following configuration:
03:26:57 DEPLOYMENT:
  DATAREADER:
    CRAWLER_ARGS: {}
    DATAREADER: DataReader
    DATASET_ARGS: {}
    GENERATOR: null
    GENERATOR_ARGS: {}
  DEPLOY: BaseDeploy
  DEPLOYMENT_ARGS: {}
  EPOCHS: 1
  OUTPUT_ARGS: {}
  PLUGIN:
    HOOKS: always
    RESET: false
EXECUTION:
  DATAREADER:
    CRAWLER_ARGS:
      file_name: original_tool_image.zip
    DATAREADER: DataReader
    DATASET_ARGS:
      annotationidx: 1
      classificationclass: vtype
      pathidx: 0
    GENERATOR: null
    GENERATOR_ARGS: {}
  EPOCHS: 10
  FP16: false
  MODEL_SERVING: Unused
  OPTIMIZER_BUILDER: ClassificationOptimizer
  PLUGIN:
    HOOKS: always
    RESET: false
  SKIPEVAL: false
  TEST_FREQUENCY: 1
  TRAINER: ClassificationTrainer
  TRAINER_ARGS:
    accumulation_steps: 4
LOGGING:
  INPUT_SIZE: null
  STEP_VERBOSE: 100
LOSS:
- KWARGS:
  - {}
  LABEL: ''
  LAMBDAS:
  - 1.0
  LOSSES:
  - Softma

46827520/46827520 bytes [████████████████████████████████████████████████████████████████████████████████████████████████████]

03:26:57 No previous stop detected. Will start from epoch 0
03:26:57 Loaded BaseStorage from ednaml.storage to build Storage
03:26:57 Reading data with DataReader DataReader
03:26:57 Default CRAWLER is <class 'ednaml.crawlers.Crawler'>
03:26:57 Default DATASET is <class 'torch.utils.data.dataset.Dataset'>
03:26:57 Default GENERATOR is <class 'ednaml.generators.ImageGenerator.ImageGenerator'>
03:26:57 Updating GENERATOR to queued class ClassificationGenerator
03:26:57 Updating CRAWLER to CarZamCrawler



Download of resnet18-5c106cde.pth to https://download.pytorch.org/models/resnet18-5c106cde.pth completed


03:26:57 Generated training data generator with 1575 training data points
03:26:57 Running classification model with classes: {'vtype': {'classes': 13}}
03:26:57 Generated test data/query generator
03:26:57 Loaded classification_model_builder from ednaml.models to build model


Outliers

types: ['ElectricVehicle', 'Wagon', 'Coupe', 'SUV', 'Van', 'PickupTruck', 'Sedan', 'Hybrid', 'LuxuryVehicle', 'Convertible', 'Diesel', 'SportsCar', 'Crossover']
Tuple_expanded

[('unzipped/original_tool_images/Hybrid Silver 2021 Toyota Avalon Hybrid.jpg', 7, 2, 0, 7), ('unzipped/original_tool_images/Crossover Black 2018 Jeep Compass.jpg', 12, 3, 5, 27), ('unzipped/original_tool_images/Sedan Gray 2017 Honda Fit.jpg', 6, 0, 8, 10), ('unzipped/original_tool_images/Pickup Truck Tan 2017 Toyota Tacoma.jpg', 5, 7, 8, 7), ('unzipped/original_tool_images/Luxury Vehicle Gold 2015 Cadillac CTS.jpg', 8, 11, 1, 18)]


03:26:58 Finished instantiating model with ClassificationResnet architecture
03:26:58 Adding plugins after constructing model
03:26:58 No saved model weights provided.
03:27:02 Model Summary retured the following error:
03:27:02 Traceback (most recent call last):
  File "/content/GLAMOR/src/ednaml/core/EdnaML.py", line 888, in getModelSummary
    self.cfg.TRAIN_TRANSFORMATION.INPUT_SIZE,
AttributeError: 'TransformationConfig' object has no attribute 'INPUT_SIZE'

03:27:02 Loaded ClassificationOptimizer from ednaml.optimizer to build Optimizer model
03:27:02 Built optimizer
03:27:02 Built scheduler
03:27:02 Added SoftmaxLogitsLoss with lambda = 1.0 and loss arguments {}
03:27:02 Built loss function
03:27:02 Built loss optimizer
03:27:02 Built loss scheduler
03:27:02 Loaded BaseStorage from ednaml.storage to build Storage
03:27:02 Loaded ClassificationTrainer from ednaml.trainer to build Trainer
03:27:02 Saving model metadata
03:27:02 Backing up metadata
03:27:02 Finished metadata backup

In [13]:
eml.train()

03:27:13 Starting training
03:27:13 Logging to:	origtoolimgs-v1-singleclass-vtype-logger.log
03:27:13 Models will be saved to local directory:	origtoolimgs-v1-singleclass-vtype
03:27:13 Models will be saved with base name:	origtoolimgs-v1_epoch[].pth
03:27:13 Optimizers will be saved with base name:	origtoolimgs-v1_epoch[]_optimizer.pth
03:27:13 Schedulers will be saved with base name:	origtoolimgs-v1_epoch[]_scheduler.pth
03:27:13 Performing initial evaluation...
03:27:20 Obtained features, validation in progress
03:27:20 Accuracy: 14.141%
03:27:20 Micro F-score: 0.141
03:27:20 Weighted F-score: 0.063
03:27:20 Starting training from 0
03:27:21 Parameter Group `opt-1`: Starting epoch 0 with 50 steps and learning rate 1.00000E-05
03:27:35 ********** Completed epoch 0 **********
03:27:35 Model evaluation triggered, but gradients still need accumulation. Will evaluate after accumulation.
03:27:35 Model save triggered, but gradients still need accumulation. Will save after accumulation.
03

In [14]:
resp = eml.eval()

03:30:32 Obtained features, validation in progress
03:30:32 Accuracy: 41.414%
03:30:32 Micro F-score: 0.414
03:30:32 Weighted F-score: 0.321


3.2 Single classification (Vehicle Color)

In [15]:
class_name = "color"   # Make sure to change this to whatever name you used for make in your `original_tool_images` crawler
class_idx = 2         # Make sure to change this to whetever index `color` is in your Crawler's tuple!
path_idx = 0          # Change this to whichever index in tuple has path
crawler_args = {"file_name" : "original_tool_image.zip"}

In [16]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [17]:
from ednaml.core import EdnaML
from ednaml.generators import ClassificationGenerator

eml = EdnaML(config = "./GLAMOR/profiles/CarZam/base_config.yml", config_inject=[
    ("SAVE.MODEL_QUALIFIER", class_name)
])

eml.cfg.EXECUTION.DATAREADER.CRAWLER_ARGS = crawler_args
eml.cfg.EXECUTION.DATAREADER.DATASET_ARGS["pathidx"] = path_idx
eml.cfg.EXECUTION.DATAREADER.DATASET_ARGS["annotationidx"] = class_idx
eml.cfg.EXECUTION.DATAREADER.DATASET_ARGS["classificationclass"] = class_name

eml.addGeneratorClass(ClassificationGenerator)
eml.addCrawlerClass(CarZamCrawler)

Injected key-value pair:  SAVE.MODEL_QUALIFIER, color


In [18]:
eml.apply()

03:35:24 ****************************************
03:35:24 
03:35:24 
03:35:24 Using the following configuration:
03:35:24 DEPLOYMENT:
  DATAREADER:
    CRAWLER_ARGS: {}
    DATAREADER: DataReader
    DATASET_ARGS: {}
    GENERATOR: null
    GENERATOR_ARGS: {}
  DEPLOY: BaseDeploy
  DEPLOYMENT_ARGS: {}
  EPOCHS: 1
  OUTPUT_ARGS: {}
  PLUGIN:
    HOOKS: always
    RESET: false
EXECUTION:
  DATAREADER:
    CRAWLER_ARGS:
      file_name: original_tool_image.zip
    DATAREADER: DataReader
    DATASET_ARGS:
      annotationidx: 2
      classificationclass: color
      pathidx: 0
    GENERATOR: null
    GENERATOR_ARGS: {}
  EPOCHS: 10
  FP16: false
  MODEL_SERVING: Unused
  OPTIMIZER_BUILDER: ClassificationOptimizer
  PLUGIN:
    HOOKS: always
    RESET: false
  SKIPEVAL: false
  TEST_FREQUENCY: 1
  TRAINER: ClassificationTrainer
  TRAINER_ARGS:
    accumulation_steps: 4
LOGGING:
  INPUT_SIZE: null
  STEP_VERBOSE: 100
LOSS:
- KWARGS:
  - {}
  LABEL: ''
  LAMBDAS:
  - 1.0
  LOSSES:
  - Softma

Outliers

types: ['ElectricVehicle', 'Wagon', 'Coupe', 'SUV', 'Van', 'PickupTruck', 'Sedan', 'Hybrid', 'LuxuryVehicle', 'Convertible', 'Diesel', 'SportsCar', 'Crossover']
Tuple_expanded

[('unzipped/original_tool_images/Hybrid Silver 2021 Toyota Avalon Hybrid.jpg', 7, 2, 0, 7), ('unzipped/original_tool_images/Crossover Black 2018 Jeep Compass.jpg', 12, 3, 5, 27), ('unzipped/original_tool_images/Sedan Gray 2017 Honda Fit.jpg', 6, 0, 8, 10), ('unzipped/original_tool_images/Pickup Truck Tan 2017 Toyota Tacoma.jpg', 5, 7, 8, 7), ('unzipped/original_tool_images/Luxury Vehicle Gold 2015 Cadillac CTS.jpg', 8, 11, 1, 18)]


03:35:25 Finished instantiating model with ClassificationResnet architecture
03:35:25 Adding plugins after constructing model
03:35:25 No saved model weights provided.
03:35:25 Model Summary retured the following error:
03:35:25 Traceback (most recent call last):
  File "/content/GLAMOR/src/ednaml/core/EdnaML.py", line 888, in getModelSummary
    self.cfg.TRAIN_TRANSFORMATION.INPUT_SIZE,
AttributeError: 'TransformationConfig' object has no attribute 'INPUT_SIZE'

03:35:25 Loaded ClassificationOptimizer from ednaml.optimizer to build Optimizer model
03:35:25 Built optimizer
03:35:25 Built scheduler
03:35:25 Added SoftmaxLogitsLoss with lambda = 1.0 and loss arguments {}
03:35:25 Built loss function
03:35:25 Built loss optimizer
03:35:25 Built loss scheduler
03:35:25 Loaded BaseStorage from ednaml.storage to build Storage
03:35:25 Loaded ClassificationTrainer from ednaml.trainer to build Trainer
03:35:25 Saving model metadata
03:35:25 Backing up metadata
03:35:25 Finished metadata backup

In [19]:
eml.train()

03:35:28 Starting training
03:35:28 Logging to:	origtoolimgs-v1-singleclass-color-logger.log
03:35:28 Models will be saved to local directory:	origtoolimgs-v1-singleclass-color
03:35:28 Models will be saved with base name:	origtoolimgs-v1_epoch[].pth
03:35:28 Optimizers will be saved with base name:	origtoolimgs-v1_epoch[]_optimizer.pth
03:35:28 Schedulers will be saved with base name:	origtoolimgs-v1_epoch[]_scheduler.pth
03:35:28 Performing initial evaluation...
03:35:30 Obtained features, validation in progress
03:35:30 Accuracy: 19.697%
03:35:30 Micro F-score: 0.197
03:35:30 Weighted F-score: 0.082
03:35:30 Starting training from 0
03:35:31 Parameter Group `opt-1`: Starting epoch 0 with 50 steps and learning rate 1.00000E-05
03:35:44 ********** Completed epoch 0 **********
03:35:44 Model evaluation triggered, but gradients still need accumulation. Will evaluate after accumulation.
03:35:44 Model save triggered, but gradients still need accumulation. Will save after accumulation.
03

In [20]:
resp = eml.eval()

03:39:17 Obtained features, validation in progress
03:39:17 Accuracy: 76.768%
03:39:17 Micro F-score: 0.768
03:39:17 Weighted F-score: 0.727
